In [ ]:
correct = ["գիրք", "տետր", "պատուհան", "Կարինե", "կարտոֆիլ", "գրիչ", "անհավանական", "քննություն", "արհեստական", "բնական", "համակարգիչ", "մատիտ", "սեղան", "գրատախտակ", "ականջակալ", "քաղաք", "հանրապետություն", "թուղթ", "գրադարան", "վարձակալություն", "գրագետ"]
correct = ["գիրտ", "տետհ", "պատուեան", "ուարինե", "կարտոֆիլ", "գրիչ", "անհավանական", "քննություն", "արհեստական", "բնական", "համակարգիչ", "մատիտ", "սեղան", "գրատախտակ", "ականջակալ", "քաղաք", "հանրապետություն", "թուղթ", "գրադարան", "վարձակալություն", "գրագետ"]


In [115]:
checker = SpellChecker()

In [116]:
len(checker.WORDS)

32198

In [117]:
checker.correction("receit")


        Edit of
            content = receit
            edit = receipt
            type = insert
            edit word probaility = 1.1653078877898144e-05
            keyboard_distance = 2
            levenshtein distance = 1
            previous edit = 
                None
        

In [118]:
candidates = list(checker.candidates("receit")) # receipt
candidates.sort(key = lambda x: checker.P(x), reverse = True)
candidates.sort(key = lambda x: x.keyboard_distance)
candidates.sort(key = lambda x: x.edit_type.value)
candidates

[
         Edit of
             content = receit
             edit = receipt
             type = insert
             edit word probaility = 1.1653078877898144e-05
             keyboard_distance = 2
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = receit
             edit = deceit
             type = replace
             edit word probaility = 3.585562731660967e-06
             keyboard_distance = 2
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = receit
             edit = recent
             type = replace
             edit word probaility = 4.750870619450781e-05
             keyboard_distance = 4
             levenshtein distance = 1
             previous edit = 
                 None
         ]

In [108]:
candidates = list(checker.candidates("receit")) # receipt
[(edit.probability(), edit.edit) for edit in candidates]

[(0.0, 'deceit'), (0.0, 'recent'), (0.0, 'receipt')]

In [110]:
checker.correction("mre")


        Edit of
            content = mre
            edit = more
            type = insert
            edit word probaility = 0.001790092193781738
            keyboard_distance = 4
            levenshtein distance = 1
            previous edit = 
                None
        

In [111]:
candidates = checker.candidates("mre") # more
candidates

{
         Edit of
             content = mre
             edit = are
             type = replace
             edit word probaility = 0.0032538981789823275
             keyboard_distance = 7
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = mre
             edit = ere
             type = replace
             edit word probaility = 8.963906829152417e-07
             keyboard_distance = 6
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = mre
             edit = ire
             type = replace
             edit word probaility = 8.963906829152417e-07
             keyboard_distance = 3
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = mre
             edit = mare
             type = insert
             edit word pro

In [87]:
checker.correction("evar")


        Edit
            content = ever
            type = replace
            previous edit = 
                None
            keyboard_distance = 3
        

In [35]:
candidates = checker.candidates("evar") # ever
candidates

{
         Edit
             content = ear
             type = delete
             previous edit = 
                 None
             keyboard_distance = 2
         ,
 
         Edit
             content = eva
             type = delete
             previous edit = 
                 None
             keyboard_distance = 0
         ,
 
         Edit
             content = ever
             type = replace
             previous edit = 
                 None
             keyboard_distance = 3
         }

In [12]:
import nltk
from nltk.util import ngrams

class NGramModel:
    
    def __init__(self, language: str):
        file_to_open = f'texts/big_{"en"}.txt'
        with open(file_to_open, "r") as file:
            corpus = file.read()
            
        tokenized_text = nltk.word_tokenize(corpus)
        self.trigrams = list(ngrams(tokenized_text, 3))
        
    def predict_next_word(self, sentence, possible_words):
        tokenized_sentence = nltk.word_tokenize(sentence)
        w1, w2 = tokenized_sentence[-2:]  # Get the last two words from the sentence

        probabilities = {}
        for trigram in self.trigrams:
            if trigram[:2] == (w1, w2):
                word = trigram[2]
                probabilities[word] = probabilities.get(word, 0) + 1

        total_count = sum(probabilities.values())
        probabilities = {word: count / total_count for word, count in probabilities.items()}
        
        if max(probabilities, key=probabilities.get) == 0:
            return None
        else:
            return max(probabilities, key=probabilities.get)


In [18]:
sentence = "I am going"
possible_words = ["to", "home", "work", "shopping"]

In [17]:
ngram_model = NGramModel("en")

In [19]:
ngram_model.predict_next_word(sentence, possible_words)

'to'

In [ ]:
sentence = "I am going to"

In [16]:
sentence.replace("I", "i")

'i am going'